<a href="https://colab.research.google.com/github/carlosnylander/TrabalhoML/blob/main/comparandoRNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Bibliotecas
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, LSTM, GRU, SimpleRNN
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt




In [ ]:
# Carregar dataset
data = pd.read_csv('/content/oficialtcld0903.csv')  # Atualize o caminho se necessário
X = data.drop(columns=['bal75'])
y = data['bal75']

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)



In [ ]:
# Modelos
def create_mlp_model(input_dim):
    model = Sequential([
        Dense(10, input_dim=input_dim, activation='relu'),
        Dense(5, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def create_cnn_model(input_dim):
    model = Sequential([
        Conv1D(10, 2, activation='relu', input_shape=(input_dim, 1)),
        Flatten(),
        Dense(10, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def create_lstm_model(input_dim):
    model = Sequential([
        LSTM(10, input_shape=(input_dim, 1), activation='tanh'),
        Dense(10, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def create_gru_model(input_dim):
    model = Sequential([
        GRU(10, input_shape=(input_dim, 1), activation='tanh'),
        Dense(10, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def create_rnn_model(input_dim):
    model = Sequential([
        SimpleRNN(10, input_shape=(input_dim, 1), activation='tanh'),
        Dense(10, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Treinamento e avaliação
def train_and_evaluate_model(model_name, model):
    if model_name in ['CNN', 'LSTM', 'GRU', 'RNN']:
        X_train_r = np.expand_dims(X_train.values, axis=2)
        X_test_r = np.expand_dims(X_test.values, axis=2)
        history = model.fit(X_train_r, y_train, epochs=50, batch_size=32, verbose=0, validation_split=0.2)
        y_pred = model.predict(X_test_r).flatten()
    else:
        history = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0, validation_split=0.2)
        y_pred = model.predict(X_test).flatten()

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f"{model_name} Model - MAE: {mae}, MSE: {mse}, RMSE: {rmse}")
    return y_pred, rmse

# Treinar modelos
mlp_model = create_mlp_model(X_train.shape[1])
cnn_model = create_cnn_model(X_train.shape[1])
lstm_model = create_lstm_model(X_train.shape[1])
gru_model = create_gru_model(X_train.shape[1])
rnn_model = create_rnn_model(X_train.shape[1])

y_pred_mlp, rmse_mlp = train_and_evaluate_model("MLP", mlp_model)
y_pred_cnn, rmse_cnn = train_and_evaluate_model("CNN", cnn_model)
y_pred_lstm, rmse_lstm = train_and_evaluate_model("LSTM", lstm_model)
y_pred_gru, rmse_gru = train_and_evaluate_model("GRU", gru_model)
y_pred_rnn, rmse_rnn = train_and_evaluate_model("RNN", rnn_model)



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


731/731 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
MLP Model - MAE: 56.638573955670225, MSE: 5808.140573012771, RMSE: 76.21115779866338
731/731 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
CNN Model - MAE: 53.130113651968514, MSE: 5051.49061878729, RMSE: 71.07383920112441
731/731 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
LSTM Model - MAE: 49.886659201833666, MSE: 4612.079273211537, RMSE: 67.91229103197401
731/731 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
GRU Model - MAE: 50.94063187768094, MSE: 4777.696194307431, RMSE: 69.12088102959504
731/731 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
RNN Model - MAE: 50.751769930925676, MSE: 4754.793998515502, RMSE: 68.95501431016821


In [ ]:
# -------------------------------------------------------------------------
# Análise de incerteza (com faixa mais escura, suavizada e robusta)
# -------------------------------------------------------------------------

# Dicionário de resultados
results = {
    "MLP": y_pred_mlp,
    "CNN": y_pred_cnn,
    "LSTM": y_pred_lstm,
    "GRU": y_pred_gru,
    "RNN": y_pred_rnn
}

# Suavização da faixa
def smooth_series(series, window_size=15):
    return pd.Series(series).rolling(window=window_size, min_periods=1, center=True).mean().values

# Avaliar Intervalos de Confiança Ajustados
def evaluate_confidence_intervals(results, y_test, scale_factor=1.0, fill_color='darkblue', alpha=0.8, smoothing=True):
    for model_name, predictions in results.items():
        print(f"\n🔎 Avaliando {model_name} com Intervalo de Confiança Ajustado...")

        std_dev = np.std(predictions)
        lower_bound = predictions - scale_factor * std_dev
        upper_bound = predictions + scale_factor * std_dev

        if smoothing:
            lower_bound = smooth_series(lower_bound)
            upper_bound = smooth_series(upper_bound)

        plt.figure(figsize=(10, 6))
        plt.plot(y_test.values[:500], 'b-', label='Real')
        plt.plot(predictions[:500], 'r-', label=f'{model_name} - Predito')
        plt.fill_between(
            range(500),
            lower_bound[:500],
            upper_bound[:500],
            color=fill_color,
            alpha=alpha,
            label=f'Intervalo (±{scale_factor*100:.0f}%)'
        )
        plt.title(f'{model_name}: Faixa de Incerteza Suavizada e Robusta')
        plt.xlabel('Índice')
        plt.ylabel('Valor')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

# Executar análise de incerteza
evaluate_confidence_intervals(
    results,
    y_test,
    scale_factor=1.0,      # Aumente/diminua para ajustar amplitude da faixa
    fill_color='darkblue', # Cor escura para visualização mais clara
    alpha=0.8,
    smoothing=True
)

In [ ]:
# Substitua pelas suas informações
username = "SEU_USUARIO"
repo_name = "NOME_DO_REPOSITORIO"
token = "SEU_TOKEN_GERADO"
email = "seuemail@exemplo.com"
nome = "Seu Nome"


In [ ]:
# Configurar identidade do git
!git config --global user.email "{email}"
!git config --global user.name "{nome}"

# Clonar o repositório com autenticação via token
!git clone https://{token}@github.com/{username}/{repo_name}.git


In [ ]:
# Seu código de análise — substitua pelo código atualizado
codigo = """
import numpy as np
import matplotlib.pyplot as plt

# Função para plotar incerteza de modelos treinados
def plotar_incerteza(y_real, y_pred, model_name, percentual=0.05):
    y_real = y_real[:500]
    y_pred = y_pred[:500]

    delta = y_pred * percentual  # ±5%
    inferior = y_pred - delta
    superior = y_pred + delta

    plt.figure(figsize=(10, 6))
    plt.plot(y_real, 'b-', label='Real')
    plt.plot(y_pred, 'r-', label='Predito')
    plt.fill_between(range(len(y_pred)), inferior, superior,
                     color='gray', alpha=0.4, label=f'Incerteza ({int(percentual*100)}%)')
    plt.title(f'{model_name}: Real vs Predito com Incerteza')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Exemplo de uso com variáveis já existentes (coloque no seu notebook depois de treinar os modelos)
# plotar_incerteza(y_test.values, y_pred_mlp, 'MLP', percentual=0.05)
# plotar_incerteza(y_test.values, y_pred_lstm, 'LSTM', percentual=0.05)
"""

# Salvando dentro do repositório clonado
with open(f"/content/{repo_name}/analise_incerteza_modelos.py", "w") as f:
    f.write(codigo)


In [ ]:
%cd /content/{repo_name}
!git add .
!git commit -m "Adiciona script de análise de incerteza com faixa ajustada"
!git push origin main  # troque 'main' por 'master' se necessário
